In [ ]:
# Inspect the input data

# from utils import plot_loss, encoder_files_to_tensors, normalize_params
# from utils import sample_files
from utils import normalizeIMG, real_files_to_tensors

import time
import glob
import tensorflow as tf
# from tensorflow import keras
import yaml
import os
import numpy as np
from datetime import datetime
import argparse
import matplotlib.pyplot as plt
import matplotlib as mpl


In [ ]:
# Initialize parameters
# data_dir = '/eos/kiliakis/tomo_data/datasets'
# data_dir = './tomo_data/datasets'
data_dir = './tomo_data/REAL_DATA_Run2'

# Data specific
IMG_OUTPUT_SIZE = 128
latent_dim = 7  # 6 + the new VrfSPS
zeropad = 14
start_turn = 1
skipturns = 3
Ib = 1.16e11



In [ ]:
wf_arr = real_files_to_tensors(data_dir)
print(wf_arr.shape)

In [ ]:
%matplotlib inline
# plot some of the outputs

nrows = 2
# Get nrows * nrows random images
sample = np.random.choice(np.arange(len(wf_arr)),
                          size=nrows * nrows, replace=False)

samples_X = tf.gather(wf_arr, sample)
# samples_X = wf_arr
# samples_y = tf.gather(y_train, sample)

# Create 3x3 grid of figures
fig, axes = plt.subplots(ncols=nrows, nrows=nrows, figsize=(12, 12))
axes = np.ravel(axes)
for i in range(len(axes)):
    ax = axes[i]
    ax.set_xticks([])
    ax.set_yticks([])
    # show the image
    ax.imshow(samples_X[i], cmap='jet')
    # Set the label
    # title = ','.join([f'{num:.1f}' for num in sample[i]])
    ax.set_title(f'{sample[i]}')

In [ ]:
# experimentation

In [ ]:
fname = os.path.join(data_dir, 'PROFILE_B2_b30660_20180908022914.npy')

with open(fname, 'rb') as f:
    timeScale_for_tomo = np.load(f)
    BunchProfiles = np.load(f)

origProfiles = BunchProfiles.copy()

fig, ax_arr = plt.subplots(ncols=3)
plt.sca(ax_arr[0])
plt.plot(timeScale_for_tomo)

plt.sca(ax_arr[1])
plt.plot(origProfiles[:, 0])

# plt.show()
# print(timeScale_for_tomo.shape)
# print(BunchProfiles.shape)

BunchProfiles = BunchProfiles*Ib/np.sum(BunchProfiles[:, 0])
plt.sca(ax_arr[2])
plt.plot(BunchProfiles[:, 0])

# print(BunchProfiles.shape)


In [ ]:
def getTimgForModelFromDataFile(BunchProfiles, T_normFactor, IMG_OUTPUT_SIZE, zeropad, start_turn, skipturns, centroid_offset=0):
    # timeScale_for_tomo, BunchProfiles = getTimeProfiles_FromData(fname, Ib)
    BunchProfiles = BunchProfiles/T_normFactor
    sel_turns = np.arange(start_turn, skipturns *
                          (IMG_OUTPUT_SIZE-2*zeropad), skipturns).astype(np.int32)
    T_img = np.pad(BunchProfiles[:, sel_turns], ((zeropad-centroid_offset, zeropad +
                                                  centroid_offset), (zeropad, zeropad)), 'constant', constant_values=(0, 0))
    # T_img_ForModel = normalizeIMG(np.reshape(T_img, T_img.shape+(1,)))
    T_img_ForModel = np.reshape(T_img, T_img.shape+(1,))
    return T_img_ForModel

origTimg = getTimgForModelFromDataFile(origProfiles, 1, IMG_OUTPUT_SIZE, zeropad, start_turn, skipturns)
adjustedTimg = getTimgForModelFromDataFile(BunchProfiles, 1, IMG_OUTPUT_SIZE, zeropad, start_turn, skipturns)



In [ ]:
print(origTimg.shape)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10, 10))
axes = np.ravel(axes)
ax = axes[0]
ax.set_xticks([])
ax.set_yticks([])
# show the image
ax.imshow(origTimg, cmap='jet')
print(np.min(origTimg), np.max(origTimg))
# Set the label
title = 'Orig'
ax.set_title(f'{title}')

ax = axes[1]
ax.set_xticks([])
ax.set_yticks([])
# show the image
ax.imshow(adjustedTimg, cmap='jet')
print(np.min(adjustedTimg), np.max(adjustedTimg))

# Set the label
title = 'Adjusted'
ax.set_title(f'{title}')

# from sklearn.preprocessing import minmax_scale
def minmax_scale(X, feature_range=(0,1)):
    min_val = np.min(X)
    max_val = np.max(X)
    scale = (feature_range[1] - feature_range[0]) / (max_val - min_val)
    return scale * (X-min_val) + feature_range[0]

ax = axes[2]
ax.set_xticks([])
ax.set_yticks([])
# show the image
origTimg_scaled = minmax_scale(origTimg.reshape(IMG_OUTPUT_SIZE, IMG_OUTPUT_SIZE))
ax.imshow(origTimg_scaled, cmap='jet')
print(np.min(origTimg_scaled), np.max(origTimg_scaled))
# Set the label
title = 'Orig_scaled'
ax.set_title(f'{title}')

ax = axes[3]
ax.set_xticks([])
ax.set_yticks([])
# show the image
adjustedTimg_scaled = minmax_scale(adjustedTimg.reshape(IMG_OUTPUT_SIZE, IMG_OUTPUT_SIZE))
ax.imshow(adjustedTimg_scaled, cmap='jet')
print(np.min(adjustedTimg_scaled), np.max(adjustedTimg_scaled))

# Set the label
title = 'Adjusted_scaled'
ax.set_title(f'{title}')